In [1]:
import sys
sys.path.append('../')
from Code.envs.SequentialMNIST import SequentialMNIST
from Code.train import train, OptWrapper
from Code.everything5 import build_standard_loop
import torch
import torch.nn as nn
from PIL import Image
import numpy as np

In [2]:
#TODO: this is same decay!!!!
MAIN_DECAY = np.exp(-1/(700)*0.5)
ADAP_DECAY = np.exp(-1/(700*2))

MAIN_DECAY = np.exp(-1/700)

spec = {
    'control_config': {
        'neuron_type': 'LIF',
        'n_neurons': 120,
        'BETA': 0.8,
        '1-beta': 'improved',
        'SPIKE_FN': 'bellec'
    },
    'mem_config': {
        'neuron_type': 'Cooldown2',
        'n_neurons': 50,
        'BETA': np.exp(-1/700),
        '1-beta': 'improved',
        'SPIKE_FN': 'bellec',
        'ADAPSCALE': 30,
        'ADAPDECAY': None, #TODO: set this
        'OFFSET': -np.log(1-np.exp(-1/700)),
        'DECAY': MAIN_DECAY
    },
    'experiment': 'SequentialMNIST',
    'lr': 0.001,
    'lr_decay': 0.9,
    'iterations': 5000,
    'batch_size': 64,
    'architecture': '1L'
}

In [3]:
DEVICE = torch.device('cuda')


train_problem = SequentialMNIST(spec['iterations'], spec['batch_size'], DEVICE, '../')
val_problem = SequentialMNIST(-1, spec['batch_size'], DEVICE, '../', validate=True)


n_in, n_out, input_rate = train_problem.get_infos()


In [4]:
from Code.everything5 import OuterWrapper, DynNetwork, ParallelNetwork2, SequenceWrapper, MeanModule, BaseNeuron, ParallelNetwork

from Code.everything5 import LIFNeuron, SeqOnlySpike, AdaptiveNeuron, CooldownNeuron, NoResetNeuron, NoResetNeuron2, NoResetNeuron3, NewFlipFlop, PositiveLinear, CooldownNeuron2
def build_standard_loop(spec, n_input, input_rate):

    neuron_lookup = {
        'LIF': LIFNeuron,
        'Disc': SeqOnlySpike,
        'Adaptive': AdaptiveNeuron,
        'Cooldown': CooldownNeuron,
        'NoReset': NoResetNeuron,
        'FlipFlop': NewFlipFlop,  # FlipFlopNeuron,
        'NoReset2': NoResetNeuron2,
        'NoReset3': NoResetNeuron3,
        'Cooldown2': CooldownNeuron2
    }

    control_neuron = neuron_lookup[spec['control_config']['neuron_type']](spec['control_config']['n_neurons'], spec['control_config'])
    mem_neuron = neuron_lookup[spec['mem_config']['neuron_type']](spec['mem_config']['n_neurons'], spec['mem_config'])
    out_neuron_size = control_neuron.out_size + mem_neuron.out_size
    out_neuron = BaseNeuron(out_neuron_size, None)

    loop_2L = {
        'input': (n_input, input_rate),
        'control': [['input', 'mem'], control_neuron, nn.Linear],
        'mem': [['control'], mem_neuron, nn.Linear],
        'output': [['control', 'mem'], out_neuron, None],
    }

    loop_1L = {
        'input': (n_input, input_rate),
        'control': [['input', 'control', 'mem'], control_neuron, nn.Linear],
        'mem': [['input', 'control', 'mem'], mem_neuron, nn.Linear],
        'output': [['control', 'mem'], out_neuron, None],
    }

    return loop_1L if spec['architecture'] == '1L' else loop_2L

loop = build_standard_loop(spec, n_in, input_rate)
loop_model = SequenceWrapper(ParallelNetwork2(loop))
out_neuron_size = loop_model.out_size

#TODO: this has to be ordered
outer = {
    'input': n_in,
    'loop': [['input'], loop_model, None],
    'mean': [['loop'], MeanModule(out_neuron_size, -56), None],
    'output': [['mean'], BaseNeuron(n_out, None), nn.Linear]
}

model = OuterWrapper(DynNetwork(outer))
model.to(DEVICE)

{'control': [[('input', 1), ('control', 1), ('mem', 1)], LIFNeuron(), <class 'torch.nn.modules.linear.Linear'>], 'mem': [[('input', 1), ('control', 1), ('mem', 1)], CooldownNeuron2(
  (elu): ELU(alpha=1.0)
), <class 'torch.nn.modules.linear.Linear'>], 'output': [[('control', 1), ('mem', 1)], BaseNeuron(), None]}


OuterWrapper(
  (model): DynNetwork(
    (layers): ModuleDict(
      (loop): SequenceWrapper(
        (model): ParallelNetwork2(
          (layers): ModuleDict(
            (control): LIFNeuron()
            (control_synapse): Linear(in_features=301, out_features=120, bias=True)
            (mem): CooldownNeuron2(
              (elu): ELU(alpha=1.0)
            )
            (mem_synapse): Linear(in_features=301, out_features=50, bias=False)
            (output): BaseNeuron()
          )
        )
      )
      (mean): MeanModule()
      (output_synapse): Linear(in_features=220, out_features=10, bias=True)
      (output): BaseNeuron()
    )
  )
)

In [5]:
optimizer = OptWrapper(model.parameters(), spec['lr'], spec['lr_decay'], 2500)

In [6]:
train(train_problem, val_problem, optimizer, model, None)

Val Acc: 8.94% | Val Time: 172.6s | Time per it: 87.8s
It:   20 | Loss: 2.317 | Acc: 10.31%
It:   40 | Loss: 2.318 | Acc: 9.69%
It:   60 | Loss: 2.297 | Acc: 12.03%
It:   80 | Loss: 2.304 | Acc: 11.80%
It:  100 | Loss: 2.261 | Acc: 14.22%
Val Acc: 15.63% | Val Time: 170.5s | Time per it: 4.4s
It:  120 | Loss: 2.177 | Acc: 15.47%
It:  140 | Loss: 2.024 | Acc: 24.06%
It:  160 | Loss: 1.907 | Acc: 31.17%
It:  180 | Loss: 1.833 | Acc: 28.98%
It:  200 | Loss: 1.838 | Acc: 30.63%
Val Acc: 25.01% | Val Time: 158.3s | Time per it: 4.1s
It:  220 | Loss: 1.880 | Acc: 26.95%
It:  240 | Loss: 1.832 | Acc: 28.83%
It:  260 | Loss: 1.808 | Acc: 31.33%
It:  280 | Loss: 1.763 | Acc: 34.69%
It:  300 | Loss: 1.734 | Acc: 34.84%
Val Acc: 34.10% | Val Time: 158.4s | Time per it: 4.1s
It:  320 | Loss: 1.753 | Acc: 32.66%
It:  340 | Loss: 1.754 | Acc: 33.44%
It:  360 | Loss: 1.879 | Acc: 27.89%
It:  380 | Loss: 1.775 | Acc: 34.30%
It:  400 | Loss: 1.794 | Acc: 29.77%
Val Acc: 33.44% | Val Time: 158.3s | Time

KeyboardInterrupt: 

In [ ]:
input = train_problem.make_inputs()#[:,:1]
out, _, log = OuterWrapper(model.model.layers.loop).to(DEVICE)(input, logging=True)
ar_length = input.shape[0]
array_list = [input, log['control'], log['mem']] #log['loop']['output']
array_list2 = []
for ar in array_list:
    array_list2.append(ar[:, 0])
    array_list2.append(torch.ones((ar_length, 1), device=input.device) * 0.5)
big_ar = (1-torch.cat(array_list2[:-1], dim=1).detach()).t() * 255
img = Image.fromarray(big_ar.cpu().numpy().astype(np.uint8), 'L')


In [ ]:


img

In [ ]:
log['control'].mean()


In [ ]:
model.model